# 算法回顾

## 监督学习算法

1. 线性回归
    - （MSE）-求值算法
    - Ridge回归
    - LASSO回归
    - 局部加权回归
    - L1，L2正则化
2. 逻辑回归（分类）
    - 梯度下降
3. SVM
    - （SMO算法）求值算法
6. 决策树
    - ID3
    - C4.5
    - CART
7. K-NN近邻算法
8. 集成学习算法
    - Bagging算法
        - 随机森林算法
    - Boosting算法
        - Adaboosting算法
        - GradientBoosting算法
            - XGBoost算法
    - Stacking算法
    
9. 贝叶斯分类算法
    - 实战

11. 神经网络（学习：AI：NN）
    - 感知器
    - 前馈神经网络FCNN
    - 卷积神经网络CNN
    - 循环神经网络RNN
    - 递归神经网络RNN
        - 长短时记忆网络
    - ......

## 无监督学习算法
4. 聚类算法：
    - K-均值算法
5. 降维与特征选择算法
    - PCA降维算法

10. 关联规则分析
    - Aprioir算法
        - 实战

## 其他算法

- 半监督学习算法
- 强化学习算法
    - 这几年比较潮流。

# 关联规则分析算法

- 场景
    - 商品推荐
        - 购买的商品->关联的商品

## 算法的思路与数据基础

- 名词；
    - 概率：概率：先验概率（边缘概率），后验概率，条件概率，似然概率，
    - 项
    - 项集
    - 频繁项集：
        - 度量：支持度（先验概率，边缘概率）
            - $P(X) = \dfrac{项集出现的次数}{总的样本数}$
        - 支持度划分频繁与非频繁：支持度阈值（用户根据场景指定）：相对度量
        
    - 关联规则
        - 度量：置信度：条件概率（似然概率）
            - $P(X | Y) =  \dfrac{P(X,Y)}{P(Y)}$
        - 关联规则的判定阈值：用户指定，根据阈值划分为
            - 强关联规则（支持度与置信度同时满足最低值）
            - 弱关联规则（如果只满足一个，或者都不满足
        - 把一个项集分成2个部分：
            - 左边部分：先导，LHS(Left Hand Side)antecedent
            - 右边部分：后继，RHS ：consequent 后件
                - 1-后继
                - 2-后继
                - n-后继

- 算法的特点：
    - 一般适合与大样本的数据集。
    - 计算量特别大。

## 关联规则分析的算法过程

### 得到所有频繁项集
- 核心：项集的拆分，计算支持度

1. 算法过程
    - k=1， 数据项集合：
        1. 构建备选项集列表：C_k；（难点）
        2. 计算C_k每个项集的支持度；
        3. 保留支持度高的项集；
        4. k = k + 1, 循环从第1步重复执行，直到C_k为空


2. 代码实现

In [2]:
# 数据集
import numpy as np
data = np.array(
    [
        ['豆奶', '莴苣'],
        ['莴苣', '尿布', '葡萄汁', '甜菜'],
        ['豆奶', '尿布', '葡萄汁', '橙汁'],
        ['莴苣', '豆奶', '尿布', '葡萄汁'],
        ['莴苣', '豆奶', '尿布', '橙汁']
    ]
)

In [7]:
# 构建集合
S =  frozenset()  # 稳定集合与非稳定集合的区别 frozenset与set的区别：
# frozenset可以作为字典的key使用，set不能作为字典的key（堆的数据无法hashable）
for item in data:
    S = S | frozenset(item)

S,type(S)
# Dic = {}
# Dic[S] = 1
# Dic

{frozenset({'尿布', '橙汁', '甜菜', '莴苣', '葡萄汁', '豆奶'}): 1}

In [23]:
# 初始化的备选项集列表C_1()直接把所有的数据做成集合，使用集合形成长度为1备选项集。
C_1 = []
for s in S:
    C_1.append(frozenset([s]))

    
# C_1 = list(map(frozenset, S))
C_1

[frozenset({'橙汁'}),
 frozenset({'莴苣'}),
 frozenset({'豆奶'}),
 frozenset({'葡萄汁'}),
 frozenset({'甜菜'}),
 frozenset({'尿布'})]

In [15]:
# 生成C_2的备箱项集（从C_1）
C_2 = []

len_C = len(C_1) 
for i in range(len_C):
    for j in range(i+1, len_C):
        C_2.append(C_1[i] |  C_1[j])


C_2       

[frozenset({'橙汁', '莴苣'}),
 frozenset({'橙汁', '豆奶'}),
 frozenset({'橙汁', '葡萄汁'}),
 frozenset({'橙汁', '甜菜'}),
 frozenset({'尿布', '橙汁'}),
 frozenset({'莴苣', '豆奶'}),
 frozenset({'莴苣', '葡萄汁'}),
 frozenset({'甜菜', '莴苣'}),
 frozenset({'尿布', '莴苣'}),
 frozenset({'葡萄汁', '豆奶'}),
 frozenset({'甜菜', '豆奶'}),
 frozenset({'尿布', '豆奶'}),
 frozenset({'甜菜', '葡萄汁'}),
 frozenset({'尿布', '葡萄汁'}),
 frozenset({'尿布', '甜菜'})]

In [24]:
# 从C2到C3，最后C4绝对不会漏掉，在C3不出现的，C4绝对不出现
# 从C2 -> C3
C_3 = []
len_C = len(C_2)
for i in range(len_C):
    c_outer = list(C_2[i])
    # 取前面2-1（3-2）
    for j in range(i+1, len_C):
        c_inner =list(C_2[j])
        
        outer_3_2 = c_outer[0:3-2]
        inner_3_2 = c_inner[0:3-2]
        
        # 为了比较的准确性，最好排序（为了防止值同，但顺序不同的情况）
        outer_3_2.sort()
        inner_3_2.sort()
        if  outer_3_2 == inner_3_2:
            # 过滤长度为是4的项集
            C_3.append(C_2[i] |  C_2[j])
        

C_3

[frozenset({'橙汁', '莴苣', '豆奶'}),
 frozenset({'橙汁', '莴苣', '葡萄汁'}),
 frozenset({'橙汁', '甜菜', '莴苣'}),
 frozenset({'尿布', '橙汁', '莴苣'}),
 frozenset({'橙汁', '葡萄汁', '豆奶'}),
 frozenset({'橙汁', '甜菜', '豆奶'}),
 frozenset({'尿布', '橙汁', '豆奶'}),
 frozenset({'橙汁', '甜菜', '葡萄汁'}),
 frozenset({'尿布', '橙汁', '葡萄汁'}),
 frozenset({'尿布', '橙汁', '甜菜'}),
 frozenset({'莴苣', '葡萄汁', '豆奶'}),
 frozenset({'甜菜', '莴苣', '葡萄汁'}),
 frozenset({'尿布', '莴苣', '葡萄汁'}),
 frozenset({'甜菜', '莴苣', '豆奶'}),
 frozenset({'尿布', '甜菜', '莴苣'}),
 frozenset({'尿布', '莴苣', '豆奶'}),
 frozenset({'甜菜', '葡萄汁', '豆奶'}),
 frozenset({'尿布', '葡萄汁', '豆奶'}),
 frozenset({'尿布', '甜菜', '豆奶'}),
 frozenset({'尿布', '甜菜', '葡萄汁'})]

In [28]:
def generate_item_sets(C_k_1, k):
    C_k = []
    len_C = len(C_k_1)
    for i in range(len_C):
        c_outer = list(C_k_1[i])
        # 取前面2-1（3-2）
        for j in range(i+1, len_C):
            c_inner =list(C_k_1[j])

            outer_k_2 = c_outer[0: k-2]
            inner_k_2 = c_inner[0: k-2]

            # 为了比较的准确性，最好排序（为了防止值同，但顺序不同的情况）
            outer_k_2.sort()
            inner_k_2.sort()
            if  outer_k_2 == inner_k_2:
                # 过滤长度为是4的项集
                C_k.append(C_k_1[i] |  C_k_1[j])
    return C_k



In [37]:
C_2 = generate_item_sets(C_1, 2)
C_3 = generate_item_sets(C_2, 3)
C_4 =  generate_item_sets(C_3, 4)
C_5 = generate_item_sets(C_4, 5)
C_6 = generate_item_sets(C_5, 6)
C_7 = generate_item_sets(C_6, 7)
C_7

[]

In [39]:
#  循环统计每个项集在总得数据集中出现的次数
data = np.array(
    [
        ['豆奶', '莴苣'],
        ['莴苣', '尿布', '葡萄汁', '甜菜'],
        ['豆奶', '尿布', '葡萄汁', '橙汁'],
        ['莴苣', '豆奶', '尿布', '葡萄汁'],
        ['莴苣', '豆奶', '尿布', '橙汁']
    ]
)

# 先原始数据转换项集的列表np.array -> list(set)
D = []
for it in data:
    D.append(frozenset(it))

D

# # 按照每个项集的列表循环
item_count = {}   # key:项集， value：出现次数的累加

for  c  in  C_3:   # 对备选项集的列表进行循环，计算每个项集的出现过数量
    for  d in D:   # 原始数据集。
        if c.issubset(d):
            # 统计（累加）
            item_count[c] = item_count.get(c, 0) + 1
    
item_count

{frozenset({'橙汁', '莴苣', '豆奶'}): 1,
 frozenset({'尿布', '橙汁', '莴苣'}): 1,
 frozenset({'橙汁', '葡萄汁', '豆奶'}): 1,
 frozenset({'尿布', '橙汁', '豆奶'}): 2,
 frozenset({'尿布', '橙汁', '葡萄汁'}): 1,
 frozenset({'莴苣', '葡萄汁', '豆奶'}): 1,
 frozenset({'甜菜', '莴苣', '葡萄汁'}): 1,
 frozenset({'尿布', '莴苣', '葡萄汁'}): 2,
 frozenset({'尿布', '甜菜', '莴苣'}): 1,
 frozenset({'尿布', '莴苣', '豆奶'}): 2,
 frozenset({'尿布', '葡萄汁', '豆奶'}): 2,
 frozenset({'尿布', '甜菜', '葡萄汁'}): 1}

In [40]:
#  循环统计每个项集在总得数据集中出现的次数
data = np.array(
    [
        ['豆奶', '莴苣'],
        ['莴苣', '尿布', '葡萄汁', '甜菜'],
        ['豆奶', '尿布', '葡萄汁', '橙汁'],
        ['莴苣', '豆奶', '尿布', '葡萄汁'],
        ['莴苣', '豆奶', '尿布', '橙汁']
    ]
)

# 先原始数据转换项集的列表np.array -> list(set)
D = []
for it in data:
    D.append(frozenset(it))

total_len = len(D)

# # 按照每个项集的列表循环
item_count = {}   # key:项集， value：出现次数的累加

for  c  in  C_3:   # 对备选项集的列表进行循环，计算每个项集的出现过数量
    for  d in D:   # 原始数据集。
        if c.issubset(d):
            # 统计（累加）
            item_count[c] = item_count.get(c, 0) + 1
    
# 支持度
item_support = {}
    
for k, v in item_count.items():
    item_support[k] = v / total_len
    
item_support

{frozenset({'橙汁', '莴苣', '豆奶'}): 0.2,
 frozenset({'尿布', '橙汁', '莴苣'}): 0.2,
 frozenset({'橙汁', '葡萄汁', '豆奶'}): 0.2,
 frozenset({'尿布', '橙汁', '豆奶'}): 0.4,
 frozenset({'尿布', '橙汁', '葡萄汁'}): 0.2,
 frozenset({'莴苣', '葡萄汁', '豆奶'}): 0.2,
 frozenset({'甜菜', '莴苣', '葡萄汁'}): 0.2,
 frozenset({'尿布', '莴苣', '葡萄汁'}): 0.4,
 frozenset({'尿布', '甜菜', '莴苣'}): 0.2,
 frozenset({'尿布', '莴苣', '豆奶'}): 0.4,
 frozenset({'尿布', '葡萄汁', '豆奶'}): 0.4,
 frozenset({'尿布', '甜菜', '葡萄汁'}): 0.2}

In [41]:
# 过滤实现
#  循环统计每个项集在总得数据集中出现的次数
data = np.array(
    [
        ['豆奶', '莴苣'],
        ['莴苣', '尿布', '葡萄汁', '甜菜'],
        ['豆奶', '尿布', '葡萄汁', '橙汁'],
        ['莴苣', '豆奶', '尿布', '葡萄汁'],
        ['莴苣', '豆奶', '尿布', '橙汁']
    ]
)

# 先原始数据转换项集的列表np.array -> list(set)
D = []
for it in data:
    D.append(frozenset(it))

total_len = len(D)

# # 按照每个项集的列表循环
item_count = {}   # key:项集， value：出现次数的累加

for  c  in  C_3:   # 对备选项集的列表进行循环，计算每个项集的出现过数量
    for  d in D:   # 原始数据集。
        if c.issubset(d):
            # 统计（累加）
            item_count[c] = item_count.get(c, 0) + 1
    
# 支持度
item_support = {}
    
for k, v in item_count.items():
    support = v / total_len
    if support >=  0.4: 
        item_support[k] = support
    
item_support

{frozenset({'尿布', '橙汁', '豆奶'}): 0.4,
 frozenset({'尿布', '莴苣', '葡萄汁'}): 0.4,
 frozenset({'尿布', '莴苣', '豆奶'}): 0.4,
 frozenset({'尿布', '葡萄汁', '豆奶'}): 0.4}

In [46]:
# 封装
def  calcute_support(D, C, threshold):
    # D: 原始数据集的项集列表的表示
    # C：备选的项集列表
    # threshold:支持度的阈值
    # 返回频繁项集列表
    # # 按照每个项集的列表循环
    item_count = {}   # key:项集， value：出现次数的累加

    for  c  in  C:   # 对备选项集的列表进行循环，计算每个项集的出现过数量
        for  d in D:   # 原始数据集。
            if c.issubset(d):
                # 统计（累加）
                item_count[c] = item_count.get(c, 0) + 1

    # 支持度
    item_support = {}

    for k, v in item_count.items():
        support = v / total_len
        if support >=  threshold: 
            item_support[k] = support

    return item_support

often_1 = calcute_support(D,C_1, 0.4)
often_2 = calcute_support(D,C_2, 0.4)
often_3 = calcute_support(D,C_3, 0.4)
often_4 = calcute_support(D,C_4, 0.4)
often_5 = calcute_support(D,C_5, 0.4)  # 原始数据中没有超过5的项集。100%返回空

{}

In [47]:
# 集成前面的函数
def generate_item_sets(C_k_1, k):
    C_k = []
    len_C = len(C_k_1)
    for i in range(len_C):
        c_outer = list(C_k_1[i])
        # 取前面2-1（3-2）
        for j in range(i+1, len_C):
            c_inner =list(C_k_1[j])

            outer_k_2 = c_outer[0: k-2]
            inner_k_2 = c_inner[0: k-2]

            # 为了比较的准确性，最好排序（为了防止值同，但顺序不同的情况）
            outer_k_2.sort()
            inner_k_2.sort()
            if  outer_k_2 == inner_k_2:
                # 过滤长度为是4的项集
                C_k.append(C_k_1[i] |  C_k_1[j])
    return C_k

def  calcute_support(D, C, threshold):
    # D: 原始数据集的项集列表的表示
    # C：备选的项集列表
    # threshold:支持度的阈值
    # 返回频繁项集列表
    # # 按照每个项集的列表循环
    item_count = {}   # key:项集， value：出现次数的累加

    for  c  in  C:   # 对备选项集的列表进行循环，计算每个项集的出现过数量
        for  d in D:   # 原始数据集。
            if c.issubset(d):
                # 统计（累加）
                item_count[c] = item_count.get(c, 0) + 1

    # 支持度
    item_support = {}

    for k, v in item_count.items():
        support = v / total_len
        if support >=  threshold: 
            item_support[k] = support

    return item_support

def get_all_often(data, support):
    all_often = {}
    # 把数据data转换为项集列表
    D = []
    for it in data:
        D.append(frozenset(it))
    
    # 先得到C1备选项集列表
    S =  frozenset()  # 稳定集合与非稳定集合的区别 frozenset与set的区别：
    # frozenset可以作为字典的key使用，set不能作为字典的key（堆的数据无法hashable）
    for item in data:
        S = S | frozenset(item)
        
    C_1 = []
    for s in S:
        C_1.append(frozenset([s]))
    # 计算C1的频繁项集列表
    O_1 = calcute_support(D, C_1, support)
    all_often.update(O_1)  # 字典的update函数
    k = 2
    O_k_1 = O_1
    while True:
        # 使用k-1大小的频繁项集列表，构建大小为k的备选项集列表C_k
        C_k = generate_item_sets(list(O_k_1.keys()), k)
        # 再产生频繁项集列表
        O_k = calcute_support(D, C_k, support)
        # 备选项集列表为空
        if len(O_k) == 0:
            break
        k += 1
        all_often.update(O_k)
        O_k_1 = O_k
    return all_often


In [52]:
data = np.array(
    [
        ['豆奶', '莴苣'],
        ['莴苣', '尿布', '葡萄汁', '甜菜'],
        ['豆奶', '尿布', '葡萄汁', '橙汁'],
        ['莴苣', '豆奶', '尿布', '葡萄汁'],
        ['莴苣', '豆奶', '尿布', '橙汁']
    ]
)
data_often = get_all_often(data, 0.4)
data_often

{frozenset({'橙汁'}): 0.4,
 frozenset({'莴苣'}): 0.8,
 frozenset({'豆奶'}): 0.8,
 frozenset({'葡萄汁'}): 0.6,
 frozenset({'尿布'}): 0.8,
 frozenset({'橙汁', '豆奶'}): 0.4,
 frozenset({'尿布', '橙汁'}): 0.4,
 frozenset({'莴苣', '豆奶'}): 0.6,
 frozenset({'莴苣', '葡萄汁'}): 0.4,
 frozenset({'尿布', '莴苣'}): 0.6,
 frozenset({'葡萄汁', '豆奶'}): 0.4,
 frozenset({'尿布', '豆奶'}): 0.6,
 frozenset({'尿布', '葡萄汁'}): 0.6,
 frozenset({'尿布', '橙汁', '豆奶'}): 0.4,
 frozenset({'尿布', '莴苣', '葡萄汁'}): 0.4,
 frozenset({'尿布', '莴苣', '豆奶'}): 0.4,
 frozenset({'尿布', '葡萄汁', '豆奶'}): 0.4}

### 找到所有关联规则
- 核心：规则拆分，计算置信度

1. 算法过程
    - 1. 找到右边的项集列表
    - 2. 利用集合的差，得到右边的项集列表（得到的项集一定是规则中出现的）
        - 右边项集列表的生成使用频繁项集。
            - 首先大小为1 - 2 - 3(1-后继，2-后继，3-后继)
    - 3. 在频繁项集列表中，查找到左边与右边的支持；
    - 4. 计算出置信度；
    - 5. 判定一下，过滤置信度低的关联规则

2. 代码实现

In [54]:
# 1. 生成1-后继（2-后继等按照前面的代码一样实现）

# 得到一个频繁项集
one_often = list(data_often.keys()) [16]
one_often

frozenset({'尿布', '葡萄汁', '豆奶'})

In [55]:
# 产生规则
# 首先构造1-后继
rhs_1 = []   # 项集的列表
for it in one_often:
    rhs_1.append(frozenset([it]))
rhs_1

[frozenset({'葡萄汁'}), frozenset({'尿布'}), frozenset({'豆奶'})]

In [64]:
# 2. 根据1-后继直积产生规则，计算置信度，过滤关联规则
rules_confidence = {}  #  key:( 先导，后继)  ，value 
# 按照后继循环生成规则
for rhs in  rhs_1:
    confidence = data_often[one_often] / data_often[rhs]
    if confidence >= 0.6:
        rules_confidence[(one_often - rhs,  rhs)] = data_often[one_often] / data_often[rhs]

rules_confidence    

{(frozenset({'尿布', '豆奶'}), frozenset({'葡萄汁'})): 0.6666666666666667}

In [69]:
# 封装
def  generate_rules_confidence(all_often, often_item, threshold):
    rhs_1 = []   # 项集的列表
    for it in often_item:
        rhs_1.append(frozenset([it]))
    
    rules_confidence = {}  #  key:( 先导，后继)  ，value 
    # 按照后继循环生成规则
    for rhs in  rhs_1:
        confidence = all_often[often_item] / all_often[rhs]
        if confidence >= threshold:
            rules_confidence[(often_item - rhs,  rhs)] = confidence
            
            

    return rules_confidence    

generate_rules_confidence(data_often, one_often, 0.5)

{(frozenset({'尿布', '豆奶'}), frozenset({'葡萄汁'})): 0.6666666666666667,
 (frozenset({'葡萄汁', '豆奶'}), frozenset({'尿布'})): 0.5,
 (frozenset({'尿布', '葡萄汁'}), frozenset({'豆奶'})): 0.5}

In [71]:
# 计算所有规则
def get_all_rule(all_often, threshold):
    all_rules = {}
    for  often_item  in  list(all_often.keys()):
        if len(often_item) >=2:
            rules_item = generate_rules_confidence(all_often,often_item, threshold)
            all_rules.update(rules_item)
    return all_rules

get_all_rule(data_often, 0.6)

{(frozenset({'豆奶'}), frozenset({'橙汁'})): 1.0,
 (frozenset({'尿布'}), frozenset({'橙汁'})): 1.0,
 (frozenset({'莴苣'}), frozenset({'豆奶'})): 0.7499999999999999,
 (frozenset({'豆奶'}), frozenset({'莴苣'})): 0.7499999999999999,
 (frozenset({'莴苣'}), frozenset({'葡萄汁'})): 0.6666666666666667,
 (frozenset({'莴苣'}), frozenset({'尿布'})): 0.7499999999999999,
 (frozenset({'尿布'}), frozenset({'莴苣'})): 0.7499999999999999,
 (frozenset({'豆奶'}), frozenset({'葡萄汁'})): 0.6666666666666667,
 (frozenset({'豆奶'}), frozenset({'尿布'})): 0.7499999999999999,
 (frozenset({'尿布'}), frozenset({'豆奶'})): 0.7499999999999999,
 (frozenset({'尿布'}), frozenset({'葡萄汁'})): 1.0,
 (frozenset({'葡萄汁'}), frozenset({'尿布'})): 0.7499999999999999,
 (frozenset({'尿布', '豆奶'}), frozenset({'橙汁'})): 1.0,
 (frozenset({'尿布', '莴苣'}), frozenset({'葡萄汁'})): 0.6666666666666667,
 (frozenset({'尿布', '豆奶'}), frozenset({'葡萄汁'})): 0.6666666666666667}

- 思考：
    - 算法还可以再优化；
    - 输出的数据格式，尽量采用主流数据格式；目前的主流：numpy.ndarray, pandas.DataFrame   
    - 去重
    - 封装成类
    - 关联规则预测
        - fit函数
        - predict(参数)
            - 商业的操作方式，训练的结果使用redis，mongoDB，mysql存储

### 应用